In [1]:
#Data preprocessing libraries
import pandas as pd  #Data manipulation
import string  #Remove punctuation & characters
import nltk  #Natural language processing 
import pickle  #For loading saved models and vectorizers
import re


from nltk.corpus import stopwords  #Stop word removal
from nltk.tokenize import word_tokenize  #Tokenizition
#from nltk.stem import PorterStemmer  #Stemming
from nltk.stem import WordNetLemmatizer  #Import WordNetLemmatizer
from nltk.corpus import wordnet  #Import WordNet

#Feature extractions libraries
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#Models libraries
from sklearn.model_selection import train_test_split #For data splitting
#Model Evaluation Function
from sklearn.metrics import accuracy_score, classification_report  #Import metrics
from sklearn.svm import SVC  #SVM Model
from sklearn.naive_bayes import MultinomialNB #Naive Bayes Model -  MultinomialNB 
from sklearn.ensemble import RandomForestClassifier #Random Forest Model

In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#Read excel file
data = pd.read_csv(r'C:\Users\s7r_2\Downloads\bbc-text (1).csv')
data.head()

NameError: name 'pd' is not defined

In [4]:
# Function to remove non-ASCII characters
def _removeNonAscii(s): 
    return "".join(i for i in s if ord(i) < 128)

# Function to clean text
def clean_text(text):
    # Ensure the input is a string
    if not isinstance(text, str):
        return ""
    
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r"what's", "what is ", text)  # Replace contractions
    text = text.replace('(ap)', '')  # Remove specific unwanted substrings
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'\W+', ' ', text)  # Replace non-word characters with a space
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r"\\", "", text)  # Remove backslashes
    text = re.sub(r"\'", "", text)  # Remove single quotes
    text = re.sub(r"\"", "", text)  # Remove double quotes
    text = re.sub('[^a-zA-Z ?!]+', '', text)  # Keep only letters and specific symbols
    text = _removeNonAscii(text)  # Remove non-ASCII characters
    text = text.strip()  # Trim leading and trailing spaces
    return text

# Function to convert a list of words into a single string
def list_to_string(tokens):
    return ' '.join(tokens)

# Ensure there is a 'text' column with tokens to clean
# Apply the cleaning function to the 'text' column and save the results in a new column
data['cleaned_text'] = data['text'].apply(clean_text)

# Display the first five rows of the DataFrame after cleaning
data.head()


NameError: name 'data' is not defined

In [26]:
# Load the list of common stopwords in English
stop_words = set(stopwords.words('english'))

# Function to tokenize text and remove stopwords
def tokenize_and_remove_stopwords(text):
    if not isinstance(text, str):  # Ensure the input is a string
        return []
    
    # Tokenize the text into individual words
    tokens = nltk.word_tokenize(text)
    # Remove stopwords from the tokens
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    
    return filtered_tokens

# Apply the tokenization and stopword removal function to the 'cleaned_text' column
data['tokens'] = data['cleaned_text'].apply(tokenize_and_remove_stopwords)
data.head()

,category,text,cleaned_text,tokens
0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"[tv, future, hands, viewers, home, theatre, sy..."
1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"[worldcom, boss, left, books, alone, former, w..."
2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"[tigers, wary, farrell, gamble, leicester, say..."
3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"[yeading, face, newcastle, fa, cup, premiershi..."
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"[ocean, twelve, raids, box, office, ocean, twe..."


In [27]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download WordNet and the POS (Part-Of-Speech) tagger data
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize WordNetLemmatizer
wn = WordNetLemmatizer()

# Function to map POS tags from NLTK to WordNet
def get_wordnet_pos(tag):
    if tag.startswith('J'):  # Adjective
        return wordnet.ADJ
    elif tag.startswith('V'):  # Verb
        return wordnet.VERB
    elif tag.startswith('N'):  # Noun
        return wordnet.NOUN
    elif tag.startswith('R'):  # Adverb
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun if no match

# Lemmatizing function with POS tagging
def lemmatizing(tokens):
    # Get POS tags for the tokens
    pos_tags = nltk.pos_tag(tokens)
    # Apply lemmatization using the POS tags
    return [wn.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags]

# Apply the lemmatizing function to the 'tokens' column
data['lemmatized'] = data['tokens'].apply(lemmatizing)
data.head()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\s7r_2\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,category,text,cleaned_text,tokens,lemmatized
0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"[tv, future, hands, viewers, home, theatre, sy...","[tv, future, hand, viewer, home, theatre, syst..."
1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"[worldcom, boss, left, books, alone, former, w...","[worldcom, bos, leave, book, alone, former, wo..."
2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"[tigers, wary, farrell, gamble, leicester, say...","[tiger, wary, farrell, gamble, leicester, say,..."
3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"[yeading, face, newcastle, fa, cup, premiershi...","[yeading, face, newcastle, fa, cup, premiershi..."
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"[ocean, twelve, raids, box, office, ocean, twe...","[ocean, twelve, raid, box, office, ocean, twel..."


In [12]:
data['lemmatized_text'] = data['lemmatized'].apply(lambda x: ' '.join(x)) #To convert the texts in the “lemmatized” column into single texts by joining the words using spaces.
data.head()


,Unnamed: 0,category,text,cleaned_text,tokens,lemmatized,lemmatized_text
0,0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,"['tv', 'future', 'hands', 'viewers', 'home', '...","['tv', 'future', 'hand', 'viewer', 'home', 'th...","[ ' t v ' , ' f u t u r e ' , ' h a n d ' ..."
1,1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldcom...,"['worldcom', 'boss', 'left', 'books', 'alone',...","['worldcom', 'bos', 'leave', 'book', 'alone', ...","[ ' w o r l d c o m ' , ' b o s ' , ' l e ..."
2,2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say th...,"['tigers', 'wary', 'farrell', 'gamble', 'leice...","['tiger', 'wary', 'farrell', 'gamble', 'leices...","[ ' t i g e r ' , ' w a r y ' , ' f a r r ..."
3,3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,"['yeading', 'face', 'newcastle', 'fa', 'cup', ...","['yeading', 'face', 'newcastle', 'fa', 'cup', ...","[ ' y e a d i n g ' , ' f a c e ' , ' n e ..."
4,4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,"['ocean', 'twelve', 'raids', 'box', 'office', ...","['ocean', 'twelve', 'raid', 'box', 'office', '...","[ ' o c e a n ' , ' t w e l v e ' , ' r a ..."


In [11]:
data.drop(columns=['lemmatized_text'], errors='ignore', inplace=True)


In [17]:
from sklearn.feature_extraction.text import CountVectorizer 

#Bag of Words (BoW)
bow_vectorizer = CountVectorizer(min_df=5)  
bow_matrix = bow_vectorizer.fit_transform(data['lemmatized'])
bow_df = pd.DataFrame.sparse.from_spmatrix(bow_matrix, columns=bow_vectorizer.get_feature_names_out())
print(bow_df)


      aaa  aaas  aaron  abandon  abandonment  abbott  abc  abide  ability  \
0       0     0      0        0            0       0    0      1        0   
1       0     0      0        0            0       0    0      0        1   
2       0     0      0        0            0       0    0      0        0   
3       0     0      0        0            0       0    0      0        0   
4       0     0      0        0            0       0    0      0        0   
...   ...   ...    ...      ...          ...     ...  ...    ...      ...   
2220    0     0      0        0            0       0    0      0        0   
2221    0     0      0        0            0       0    0      0        0   
2222    0     0      0        0            0       0    0      0        0   
2223    0     0      0        0            0       0    0      0        0   
2224    0     0      0        0            0       0    0      0        0   

      able  ...  zach  zealand  zeppelin  zero  zhang  zimbabwe  zombie  zo

In [2]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#TF-IDF
tfidf_BBC = TfidfVectorizer(min_df=1)#Ignore rare words
tfidf_matrix = tfidf_BBC.fit_transform(data['lemmatized'])#tfidf_matrix-Represents the weight of each word based on its frequency.
#Fit_transform-To convert texts to a matrix.

#Convert to DataFrame to display the data more clearly
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_BBC.get_feature_names_out())

print("\nTF-IDF Matrix:")
print(tfidf_df)
with open("tfidf_BBC.pickle", "wb") as file:
    pickle.dump(tfidf_BBC, file)

NameError: name 'data' is not defined

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import pickle
# Prepare data for partitioning
X = tfidf_matrix
y = data['category'] 

#Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size=0.2, random_state=42)  #Split data

#Function to evaluate model performance
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)  #Make predictions
    accuracy = accuracy_score(y_test, predictions)  #Calculate accuracy
    report = classification_report(y_test, predictions)  #Generate classification report
    return accuracy, report

svm_model = SVC(kernel='linear', class_weight='balanced')  #Initialize SVM model with linear kernel and class weights
svm_model.fit(X_train, y_train)  #Train the model
svm_accuracy, svm_report = evaluate_model(svm_model, X_test, y_test)  #Evaluate SVM
print("SVM Accuracy:", svm_accuracy)  #Print accuracy
print("SVM Classification Report:\n", svm_report)  #Print classification report

nb_model = MultinomialNB()  #Initialize Naive Bayes model
nb_model.fit(X_train, y_train)  #Train the model
nb_accuracy, nb_report = evaluate_model(nb_model, X_test, y_test)  #Evaluate Naive Bayes
print("Naive Bayes Accuracy:", nb_accuracy)  #Print accuracy
print("Naive Bayes Classification Report:\n", nb_report)  #Print classification report

rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)  #Initialize Random Forest model with class weights
rf_model.fit(X_train, y_train)  #Train the model
rf_accuracy, rf_report = evaluate_model(rf_model, X_test, y_test)  #Evaluate Random Forest
print("Random Forest Accuracy:", rf_accuracy)  #Print accuracy
print("Random Forest Classification Report:\n", rf_report)  #Print classification report
with open("model_bbc.pickle", "wb") as model_file:
    pickle.dump(svm_model, model_file)

SVM Accuracy: 0.9685393258426966
SVM Classification Report:
                precision    recall  f1-score   support

     business       0.96      0.92      0.94       101
entertainment       0.98      0.99      0.98        81
     politics       0.94      0.96      0.95        83
        sport       0.98      1.00      0.99        98
         tech       0.99      0.98      0.98        82

     accuracy                           0.97       445
    macro avg       0.97      0.97      0.97       445
 weighted avg       0.97      0.97      0.97       445

Naive Bayes Accuracy: 0.9573033707865168
Naive Bayes Classification Report:
                precision    recall  f1-score   support

     business       0.94      0.94      0.94       101
entertainment       1.00      0.89      0.94        81
     politics       0.92      0.98      0.95        83
        sport       0.99      1.00      0.99        98
         tech       0.94      0.98      0.96        82

     accuracy                   

In [25]:
import pickle
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Function to preprocess text data
def preprocess_data(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s\'\"]', '', text)  # Remove punctuation
    return text

# Ensure there are no null or empty texts in the dataset
data = data[data['lemmatized'].notnull() & (data['lemmatized'] != '')]

# Preprocess the lemmatized text
data['lemmatized_text'] = data['lemmatized'].apply(preprocess_data)

# Create a TF-IDF matrix
tfidf_BBC = TfidfVectorizer(min_df=5)  # Minimum document frequency of 5
tfidf_matrix = tfidf_BBC.fit_transform(data['lemmatized_text'])

# Prepare feature matrix (X) and target vector (y)
X = tfidf_matrix
y = data['category']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Logistic Regression model
lg_model = LogisticRegression(max_iter=200, verbose=1)  # Increase max iterations for convergence
lg_model.fit(X_train, y_train)

# Save the model and the TF-IDF vectorizer
with open("model_bbc.pickle", "wb") as model_file:
    pickle.dump(lg_model, model_file)

with open("tfidf_BBC.pickle", "wb") as file:
    pickle.dump(tfidf_BBC, file)

# Function to predict the category of a given text
def predict_category(text):
    cleaned_text = preprocess_data(text)  # Preprocess the input text
    tfidf_data = tfidf_BBC.transform([cleaned_text])  # Convert text to TF-IDF features
    prediction = lg_model.predict(tfidf_data)  # Predict category using the model
    return prediction[0]  # Return the predicted category

# Test the prediction function
new_text = "worldcom boss left books alone former worldcom boss bernie ebbers who is accused of overseeing an $11bn"
predicted_category = predict_category(new_text)
print(f"The predicted category is: {predicted_category}")


The predicted category is: business


In [1]:
import os
import sys
import pickle
from pathlib import Path
import tkinter as tk
from tkinter import messagebox

# Load the trained model and TF-IDF vectorizer
with open("model_bbc.pickle", "rb") as model_file:
    svm_model = pickle.load(model_file)

with open("tfidf_BBC.pickle", "rb") as file:
    tfidf_BBC = pickle.load(file)

# Function to preprocess text
def preprocess_data(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'[^\w\s\'\"]', '', text)  # Remove punctuation
    return text

# Function to classify the text using the loaded model
def predict_category(text):
    cleaned_text = preprocess_data(text)  # Preprocess the input text
    tfidf_data = tfidf_BBC.transform([cleaned_text])  # Convert text to TF-IDF features
    prediction = svm_model.predict(tfidf_data)  # Predict the category
    return prediction[0]  # Return the predicted category

# Function to classify the entered text and display the result
def classify_text():
    text = entry.get()  # Get the input text from the entry field
    if not text.strip():  # Check if the text is empty
        messagebox.showwarning("Error", "Please enter text for classification!")  # Show a warning
        return
    category = predict_category(text)  # Predict the category
    result_label.config(text=f"Type of news: {category}")  # Display the result

# Function to clear the input text and reset the result label
def clear_text():
    entry.delete(0, tk.END)  # Clear the text entry
    result_label.config(text="")  # Reset the result label

# Set up the main GUI window
root = tk.Tk()
root.geometry("800x500")  # Set the window size
root.title("BBC News Classification")  # Set the window title

# Add GUI elements
label1 = tk.Label(root, text="News Classification Analyst", font=("Helvetica", 14))  # Main title
label1.pack(pady=10)

label2 = tk.Label(root, text="Enter text:", font=("Helvetica", 12))  # Instruction label
label2.pack(pady=5)

entry = tk.Entry(root, font=("Helvetica", 12), width=50)  # Text entry field
entry.pack(pady=5)

classify_button = tk.Button(root, text="Classification", command=classify_text, font=("Helvetica", 12), bg="#002179", fg="white")
classify_button.pack(pady=10)  # Button to classify text

clear_button = tk.Button(root, text="Clear Text", command=clear_text, font=("Helvetica", 12), bg="#d9534f", fg="white")
clear_button.pack(pady=5)  # Button to clear text

result_label = tk.Label(root, text="", font=("Helvetica", 16), fg="blue")  # Label to display the classification result
result_label.pack(pady=20)

# Run the GUI application
root.mainloop()
